In [ ]:
import altair as alt
import pandas as pd
from vega_datasets import data

# 1. Color the lines of a slopegraph, based on one of the fields, if the slope is positive.

Source: https://github.com/altair-viz/altair/issues/2178 and https://altair-viz.github.io/gallery/slope_graph.html.

In [ ]:
source = data.barley()

alt.Chart(source).mark_line().encode(
    x="year:O",
    y="median_yield:Q",
    color=alt.condition("datum.slope > 0", "site:N", alt.value("lightgray")),
).transform_aggregate(
    median_yield="median(yield)", groupby=["year", "site"]
).transform_pivot(
    "year", value="median_yield", groupby=["site"]
).transform_calculate(
    slope=alt.datum["1932"] - alt.datum["1931"]
).transform_fold(
    ["1931", "1932"], as_=["year", "median_yield"]
)

# 2. Layered count histogram and cumulative histogram.

Source: https://vega.github.io/vega-lite/examples/layer_cumulative_histogram.html.

In [ ]:
movies = data.movies()
movies.head()

In [ ]:
base = (
    alt.Chart(movies)
    .encode(
        x=alt.X("bin_IMDB_Rating:Q", axis=alt.Axis(title="IMDB Rating")),
        x2=alt.X2("bin_IMDB_Rating_end:Q"),
    )
    .transform_bin("bin_IMDB_Rating", field="IMDB Rating", bin=True)
    .transform_aggregate(
        groupby=["bin_IMDB_Rating", "bin_IMDB_Rating_end"], count="count()",
    )
    .transform_filter("datum.bin_IMDB_Rating != null")
    .transform_window(
        cumulative_count="sum(count)", sort=[alt.SortField("bin_IMDB_Rating")]
    )
)

count_hist = base.mark_bar(color="yellow", opacity=0.5).encode(y=alt.Y("count:Q"))

cum_hist = base.mark_bar().encode(y=alt.Y("cumulative_count:Q"))

cum_hist + count_hist

# 3. Chart title with the same style as the axis title or axis labels (via built-in guide styles).

Source: https://altair-viz.github.io/user_guide/generated/core/altair.TitleParams.html and https://vega.github.io/vega-lite/docs/config.html#guide-config

In [ ]:
source = alt.pd.DataFrame(
    {
        "a": ["A", "B", "C", "D", "E", "F", "G", "H", "I"],
        "b": [28, 55, 43, 91, 81, 53, 19, 87, 52],
    }
)

default = (
    alt.Chart(source, title=alt.TitleParams("Bar Chart", style="group-title"))
    .mark_bar()
    .encode(x="a", y="b")
)

axis_title = (
    alt.Chart(source, title=alt.TitleParams("Bar Chart", style="guide-title"))
    .mark_bar()
    .encode(x="a", y="b")
)

axis_labels = (
    alt.Chart(source, title=alt.TitleParams("Bar Chart", style="guide-label"))
    .mark_bar()
    .encode(x="a", y="b")
)

default | axis_title | axis_labels